# Connect Playlists and Locations

In [66]:
import pandas as pd
import time
import json
import googlemaps
import homebrew as hb
from sklearn.neighbors import NearestNeighbors

In [69]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
df = pd.read_json('artists_shows.json', lines=True)

In [5]:
print(df.shape)
df.head()

(134943, 3)


,artist,date,loc
0,2 Chainz,2019-06-30 05:30:00,"Las Vegas, NV, US"
1,2 Chainz,2019-07-12 11:00:00,"London, UK"
2,2 Chainz,2019-08-10 00:00:00,"Anaheim, CA, US"
3,2 Chainz,2019-08-24 22:00:00,"Atlanta, GA, US"
4,2 Chainz,2019-05-27 05:30:00,"Las Vegas, NV, US"


In [34]:
df.date.describe()

count                  134943
unique                  37136
top       2019-07-19 00:00:00
freq                      191
first     1973-09-29 00:00:00
last      2020-12-16 18:30:00
Name: date, dtype: object

In [56]:
df.columns = ['artist', 'date', 'location']    # rename loc -> location
domestics = df[df['location'].str.endswith('US')]    # drop any foreign gigs
recent_domestics = domestics[domestics['date'] >= '2015']    #  drop any gigs older than 5 yrs
locations = recent_domestics.copy()

In [59]:
unique_locations = locations['location'].unique()
f'There are {unique_locations.shape[0]} unique locations'

'There are 2023 unique locations'

Get lat longs from googlemaps

In [53]:
# unique_loc_w_latlong = []

# for i, location in enumerate(unique_locations):
#     row = {}
#     lat, lng = hb.get_latlong(location)
#     row['location'] = location
#     row['lat'] = lat
#     row['lng'] = lng
#     unique_loc_w_latlong.append(row)
#     if i % 50 == 0:
#         time.sleep(2)
#     if i % 100 == 0:
#         print(i)                # Slowish 

In [60]:
latlongs_df = pd.DataFrame(unique_loc_w_latlong)
latlongs_df.head()

,lat,lng,location
0,36.169941,-115.139830,"Las Vegas, NV, US"
1,33.836593,-117.914301,"Anaheim, CA, US"
2,33.748995,-84.387982,"Atlanta, GA, US"
3,42.331427,-83.045754,"Detroit, MI, US"
4,40.691613,-112.001050,"West Valley City, UT, US"


In [61]:
shows_w_latlong = pd.merge(locations, latlongs_df, on='location', how='left')
print(locations.shape)
shows_w_latlong.shape

(75699, 3)


(75699, 5)

In [62]:
shows_w_latlong.to_json('shows_w_latlong.json')

In [65]:
artists_playlists = pd.read_json('trimmed_artists_w_flags.json', lines=True)

In [73]:
similar_artist_model = hb.SimilarArtistModel(artists_playlists)